In this notebook, we try to demonstrate the basic usage of the evolutionary parameter grid. Similar to the parameter grid in the "Scikit-learn" package, the basic usage between two packages is basically the same. However, in contrast to enumerate all possible combinations, this package uses the genetic algorithm to solve the unsolvable combinatorial optimization problem which may produce large grid search.

Although there are a lot of evolutionary algorithm based parameter packages, most of them are only confined to the hyper-parameter search problem, and only few of them can be used as a general combinatorial optimization problem solver. Thus, this package provides a flexible combinatorial optimization problem solver, which can be easily used on various combinatorial optimization problems, such as hyper-parameter selection, feature selection, or even neural architecture search, as long as the parameter grid is well designed.

In this tutorial, we use the hyper-parameter searching task as an example to demonstrate the usage of this package.

Firstly, we need to load the experimental dataset and to define the search space.

In [1]:
import random
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score, ParameterGrid, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from EvolutionaryParameterGrid import EAParameterGrid

random.seed(0)
np.random.seed(0)

X, y = load_boston(return_X_y=True)
param_grid = {
    "max_depth": np.random.randint(1, (X.shape[1] * .85), 20),
    "max_features": np.random.randint(1, X.shape[1], 20),
    "min_samples_leaf": [2, 3, 4, 5, 6],
    "criterion": ["mse", "mae"],
}

After loaded the experimental dataset, we can use the evolutionary parameter grid just as a traditional parameter grid. However, it should be note that, due to evolutionary algorithm need to intelligent tune it's exploration proposal based on the fitness score, i.e. , the cross validation score in our case. Thus, we need to manually set the fitness score after getting the cross validation score.

In [2]:
grid = EAParameterGrid()
for param in grid.grid(param_grid):
    tree = DecisionTreeRegressor(**param,random_state=0)
    score = cross_val_score(tree, X, y, cv=5)
    grid.set_fitness(np.mean(score))

gen	nevals	avg     	std     	min     	max     
0  	50    	0.172936	0.427611	-2.23543	0.521039
1  	50    	0.397736	0.11816 	0.0286067	0.522334
2  	50    	0.476128	0.101181	0.156791 	0.522334
3  	50    	0.48994 	0.0817068	0.240743 	0.522334
4  	50    	0.487414	0.0961236	0.133501 	0.522334
5  	50    	0.489379	0.123043 	-0.290536	0.522334
6  	50    	0.498634	0.0721583	0.189493 	0.522334
7  	50    	0.495893	0.090845 	0.0234607	0.522334
8  	50    	0.484376	0.0968043	0.11605  	0.522334
9  	50    	0.507147	0.0487307	0.268768 	0.522334
10 	50    	0.490803	0.115769 	-0.104581	0.522334
11 	50    	0.500932	0.136558 	-0.450181	0.522334
12 	50    	0.480806	0.142026 	-0.364595	0.522334
13 	50    	0.51087 	0.0488582	0.240743 	0.522334
14 	50    	0.515311	0.0377774	0.268768 	0.522334
15 	50    	0.510662	0.0568323	0.13672  	0.522334
16 	50    	0.510165	0.0621671	0.116765 	0.522334
17 	50    	0.508568	0.0813688	-0.0523502	0.522334
18 	50    	0.506901	0.060026 	0.189493  	0.522334
19 	50    	0.509815	0.04

It should be noted that, although we set a huge population size and a huge generation number, that does not mean that we really need to evaluate such many proposals. Because we use a memory mechanism in our package, we only need to evaluate a small fraction of those proposals, because most of them are repetitive proposals. Therefore, we use the following command to examine the real evaluation times.

In [3]:
grid.history_dict.__len__()

200

Well, it seems that we only use 200 times to try different parameter combinations. Is that really useful? We can compare it with the well-known random search method under the same evaluation times.

In [4]:
cv = RandomizedSearchCV(DecisionTreeRegressor(random_state=0),
                        param_grid,
                        n_iter=200,
                        cv=5,
                        verbose=1)
cv.fit(X, y)
cv.best_score_

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    4.3s finished


0.5098659816094302

From the result, we find that the evolutionary based method is more efficient compared with the random search method. This is not out of our surprise because it has been proved in many applications. Thus, we highly recommend using evolutionary based search as an enhancement method when there is no available algorithm other than random search, because it often can achieve an unexpected high performance.